# Tutorial

We are going to use [Simple Transformers](https://github.com/ThilinaRajapakse/simpletransformers) - an NLP library based on the [Transformers](https://github.com/huggingface/transformers) library by HuggingFace. Simple Transformers allows us to fine-tune Transformer models in a few lines of code.  

As the dataset, we are going to use the [Germeval 2019](https://projects.fzai.h-da.de/iggsa/projekt/), which consists of German tweets. We are going to detect and classify abusive language tweets. These tweets are categorized in 4 classes: `PROFANITY`, `INSULT`, `ABUSE`, and `OTHERS`. The highest score achieved on this dataset is `0.7361`.

### We are going to

- install Simple Transformers library
- select a pre-trained monolingual model
- load the dataset
- train/fine-tune our model
- evaluate the results of it
- save and load the model
- test the loaded model on a real example

# Install Simple Transformers library 

In [ ]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

simpletransformers==0.40.0


# Select a pre-trained monolingual model

As mentioned above the Simple Transformers library is based on the Transformers library from HuggingFace. This enables us to use every pre-trained model provided in the [Transformers library](https://huggingface.co/transformers/pretrained_models.html) and all community-uploaded models. For a list that includes community-uploaded models, refer to [https://huggingface.co/models](https://huggingface.co/models).

We are going to use the `distilbert-base-german-cased` model. [DistilBERT is a small, fast, cheaper version of BERT](https://huggingface.co/transformers/model_doc/distilbert.html). It has 40% less parameters than `bert-base-uncased` and runs 60% faster while preserving over 95% of Bert’s performance.

# Load the dataset

In [ ]:
!wget https://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/08/germeval2019GoldLabelsSubtask1_2.txt
!wget https://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/09/germeval2019.training_subtask1_2_korrigiert.txt

--2020-06-24 09:19:44--  https://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/08/germeval2019GoldLabelsSubtask1_2.txt
Resolving projects.fzai.h-da.de (projects.fzai.h-da.de)... 141.100.60.75, 2001:67c:2184:82a:21a:4aff:fe16:1e6
Connecting to projects.fzai.h-da.de (projects.fzai.h-da.de)|141.100.60.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 543975 (531K) [text/plain]
Saving to: ‘germeval2019GoldLabelsSubtask1_2.txt.1’

germeval2019GoldLab 100%[===================>] 531.23K   406KB/s    in 1.3s    

2020-06-24 09:19:47 (406 KB/s) - ‘germeval2019GoldLabelsSubtask1_2.txt.1’ saved [543975/543975]

--2020-06-24 09:19:48--  https://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/09/germeval2019.training_subtask1_2_korrigiert.txt
Resolving projects.fzai.h-da.de (projects.fzai.h-da.de)... 141.100.60.75, 2001:67c:2184:82a:21a:4aff:fe16:1e6
Connecting to projects.fzai.h-da.de (projects.fzai.h-da.de)|141.100.60.75|:443... connected.
HTTP request sent, a

In [ ]:
import pandas as pd

class_list = ['INSULT','ABUSE','PROFANITY','OTHER']

df1 = pd.read_csv('germeval2019GoldLabelsSubtask1_2.txt',sep='\t', lineterminator='\n',encoding='utf8',names=["tweet", "task1", "task2"])
df2 = pd.read_csv('germeval2019.training_subtask1_2_korrigiert.txt',sep='\t', lineterminator='\n',encoding='utf8',names=["tweet", "task1", "task2"])

df = pd.concat([df1,df2])
df['task2'] = df['task2'].str.replace('\r', "")
df['pred_class'] = df.apply(lambda x:  class_list.index(x['task2']),axis=1)


df = df[['tweet','pred_class']]

print(df.shape)
df.head()

(7011, 2)


,tweet,pred_class
0,@JanZimmHHB @mopo Komisch das die Realitätsver...,0
1,@faznet @Gruene_Europa @SPDEuropa @CDU CDU ste...,1
2,"@DLFNachrichten Die Gesichter, Namen, Religion...",3
3,@welt Wie verwirrt muss man sein um sich zu we...,1
4,@hacker_1991 @torben_braga Weil die AfD den Fe...,1


In [ ]:
print(df2)

                                                  tweet    task1        task2
0     @jouwatch Hat die Polizei keine Kanone mehr ? ...  OFFENSE      ABUSE\r
1     @de_sputnik @Saudi Arabien habt ihr mal wieder...  OFFENSE      ABUSE\r
2     Glaube ich nicht , die Bundesregierung so wie ...  OFFENSE      ABUSE\r
3      Doch schockierend viele Jugendliche wissen ka...  OFFENSE  PROFANITY\r
4     Sein Charakter war ihm wichtiger anstatt als b...  OFFENSE  PROFANITY\r
...                                                 ...      ...          ...
3975  250 Menschen auf der Demonstration gegen das D...    OTHER      OTHER\r
3976  Erneut Massaker an Kurdische ZivilistInnen dur...    OTHER      OTHER\r
3977  Hunderte Refugees haben die Grenze zur spanisc...    OTHER      OTHER\r
3978  Heute ab 17:00 Uhr an der Alten Oper FFM: Kund...    OTHER      OTHER\r
3979  Uns gibt es jetzt auch auf #twitter. Folgt uns...    OTHER      OTHER\r

[3980 rows x 3 columns]


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.10)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (6309, 2)
test shape:  (702, 2)


# Load pre-trained model

In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "distilbert-base-german-cased",
    num_labels=4,
    args=train_args
)

# Train model

In [ ]:
# Train the model
model.train_model(train_df)



RuntimeError: ignored

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

# save and load the model

save files without outputs/ 


In [ ]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [ ]:
save_model('outputs','germeval-distilbert-german')

In [ ]:
!tar -zxvf ./germeval-distilbert-german.tar.gz

In [ ]:
!rm -rf outputs

# Test the loaded model on a real example

In [ ]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('germeval-distilbert-german')

In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "outputs/",
    num_labels=4,
    args=train_args
)

In [ ]:
class_list = ['INSULT','ABUSE','PROFANITY','OTHER']

test_tweet = "Meine Mutter hat mir erzählt, dass mein Vater einen Wahlkreiskandidaten nicht gewählt hat, weil der gegen die Homo-Ehe ist"

predictions, raw_outputs = model.predict([test_tweet])

print(class_list[predictions[0]])

In [ ]:
class_list = ['INSULT','ABUSE','PROFANITY','OTHER']

test_tweet = "Frau #Böttinger meine Meinung dazu ist sie sollten uns mit ihrem Pferdegebiss nicht weiter belästigen #WDR"

predictions, raw_outputs = model.predict([test_tweet])

print(class_list[predictions[0]])
# INSULT